# Batch training & tuning on Ray Tune

Batch training and tuning are common tasks in simple machine learning use-cases such as time series forecasting. They require fitting of simple models on multiple data batches corresponding to locations, products, etc.

In the context of this notebook, batch training is understood as creating the same model(s) for different and separate data, or subsets of a dataset. This notebook showcases how to conduct batch training using [Ray Tune](https://docs.ray.io/en/latest/tune/index.html).

![Batch training diagram](../../data/examples/images/batch-training.svg)

For the data, we will use the [NYC Taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). This popular tabular dataset contains historical taxi pickups by timestamp and location in NYC. To demonstrate batch training, we will simplify the data to a regression problem to predict `trip_duration` and use scikit-learn.

To demonstrate how batch training can be parallelized, we will train a separate model for each dropoff location. This means we can use the `dropoff_location_id` column in the dataset to group the dataset into data batches. Then we will fit a separate model for each batch and evaluate it.

# Introduction to Ray Tune

> **While Tune's main purpose is hyperparameter optimization, you can also use it as an execution engine to manage multiple independent compute workers.**  

In this notebook, we will use [Ray Tune](https://docs.ray.io/en/latest/tune/index.html) to run separate, parallel training jobs, each as a different experiment, per dropoff location.  After all the Tune experiments have run, we will pick the best model per dropoff location.  

Let us quickly walk through the [key concepts](https://docs.ray.io/en/latest/tune/key-concepts.html) you need to know to use Tune.

- First, you define a `Search space` and pass that into a `Trainable` or `callable` function, that specifies the objective you want to tune.  The trainable function will be called for every experiment in the search space.

- Then you select a `Search algorithm` and optionally use a
scheduler to stop searches early and speed up your experiments.  We will use simple **grid search** (run every permutation as a separate trial) as our search algorithm in this tutorial.

- Together with other configurations, the Trainable, Search algorithm, and Scheduler are passed into `Tuner`, which runs your experiments in parallel.

- The output from `Tuner.fit()` can be analyzed to inspect your experiment results.  The following figure shows an overview of the Ray Tune flow, which we will use in this tutorial.

![Tune flow diagram](../../tune/images/tune_flow.png)

# Contents

In this this tutorial, you will learn about:
 1. [Define how to load and prepare Parquet data](#load_data)
 2. [Define your Ray Tune Search Space and Search Algorithm](#define_search_space)
 3. [Define a Trainable (callable) function](#define_trainable)
 4. [Run independent experiments in Parallel using Ray AIR Tune](#run_tune_search)
 5. [Load a model from checkpoint and perform inference](#load_checkpoint)

# Walkthrough

Let us start by importing a few required libraries, including open-source [Ray](https://github.com/ray-project/ray) itself!

In [1]:
import os
print(f'Number of CPUs in this system: {os.cpu_count()}')
from typing import Tuple, List, Union, Optional, Callable
import time
import pandas as pd
import numpy as np
import pyarrow
import pyarrow.parquet as pq
import pyarrow.dataset as pds
print(f"pyarrow: {pyarrow.__version__}")

Number of CPUs in this system: 8
pyarrow: 6.0.1


In [2]:
import ray

if ray.is_initialized():
    ray.shutdown()
ray.init()

2022-11-09 11:17:36,892	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.13
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


In [3]:
print(ray.cluster_resources())

{'CPU': 8.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 2147483648.0, 'memory': 8644909466.0}


<b>Scikit-Learn</b> is one of the most widely used tools in the ML community for working with data, offering dozens of easy-to-use machine learning algorithms. 
> However, to achieve high performance, you often need to perform model selection and checkpointing. 

[Train-sklearn libraries](https://docs.ray.io/en/latest/tune/examples/tune-sklearn.html) make these tasks more convenient in a distributed setting.  Below, we import sklearn, ray Tune and AIR, and our custom Train-sklearn libraries.


In [4]:
# import standard sklearn libraries
import sklearn
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
print(f"sklearn: {sklearn.__version__}")

# import ray libraries
from ray import air, tune
from ray.air import session
from ray.train.sklearn import SklearnCheckpoint
from ray.train.sklearn import SklearnPredictor

# set global random seed for sklearn models
np.random.seed(415)

sklearn: 1.1.2


In [5]:
# For benchmarking purposes, we can print the times of various operations.
# In order to reduce clutter in the output, this is set to False by default.
PRINT_TIMES = False

def print_time(msg: str):
    if PRINT_TIMES:
        print(msg)
        
# To speed things up, we’ll only use a small subset of the full dataset consisting of two last months of 2019.
# You can choose to use the full dataset for 2018-2019 by setting the SMOKE_TEST variable to False.
SMOKE_TEST = True


## Define how to load and prepare Parquet data <a class="anchor" id="load_data"></a>

First, we need to load some data.  Since the NYC Taxi dataset is fairly large, we will filter files first into a PyArrow dataset. And then in the next cell after, we will filter the data on read into a PyArrow table and convert that to a pandas dataframe.

```{tip}
Use PyArrow dataset and table for reading or writing large parquet files, since its native multithreaded C++ adpater is faster than pandas read_parquet, even using engine=pyarrow.
```

In [6]:
# Define some global variables.
target = "trip_duration"
s3_partitions = pds.dataset(
    "s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/",
    partitioning=["year", "month"],
)
s3_files = [f"s3://{file}" for file in s3_partitions.files]

# Obtain all location IDs
all_location_ids = (
    pq.read_table(s3_files[0], columns=["dropoff_location_id"])["dropoff_location_id"]
    .unique()
    .to_pylist()
)

# Use smoke testing or not.
starting_idx = -1 if SMOKE_TEST else 0
sample_locations = [145, 166, 152] if SMOKE_TEST else all_location_ids

# Display what data will be used.
s3_files = s3_files[starting_idx:]
print(f"NYC Taxi using {len(s3_files)} file(s)!")
print(f"s3_files: {s3_files}")
print(f"Locations: {sample_locations}")


NYC Taxi using 1 file(s)!
s3_files: ['s3://air-example-data/ursa-labs-taxi-data/by_year/2019/06/data.parquet/ab5b9d2b8cc94be19346e260b543ec35_000000.parquet']
Locations: [145, 166, 152]


In [7]:
# Function to read a pyarrow.Table object using pyarrow parquet 
def read_data(file: str, sample_id: np.int32) -> pd.DataFrame:
    
    df = pq.read_table(
        file,
        filters=[
            ("passenger_count", ">", 0),
            ("trip_distance", ">", 0),
            ("fare_amount", ">", 0),
            ("pickup_location_id", "in", [264, 265]),
            ("dropoff_location_id", "not in", [264, 265]), 
            ("dropoff_location_id", "=", sample_id)
        ],
        columns=[
            "pickup_at",
            "dropoff_at",
            "pickup_location_id",
            "dropoff_location_id",
            "passenger_count",
            "trip_distance",
            "fare_amount",
        ],
    ).to_pandas()

    return df

# Function to transform a pandas dataframe
def transform_df(the_df: pd.DataFrame) -> pd.DataFrame:
    df = the_df.copy()
    
    df["trip_duration"] = (df["dropoff_at"] - df["pickup_at"]).dt.seconds
    df = df[df["trip_duration"] > 60]
    df = df[df["trip_duration"] < 24 * 60 * 60] 
    df.drop(["dropoff_at", "pickup_at", "pickup_location_id", "fare_amount"]
            , axis=1, inplace=True)
    df["dropoff_location_id"] = df["dropoff_location_id"].fillna(-1)
    return df

## Define your Ray Tune Search Space and Search Algorithm <a class="anchor" id="define_search_space"></a>

**First, define a search space of experiment to run.** 
> The search space together with a search algorithm determine which experiments will be run.  

Common search algorithms include grid search, random search, Bayesian search, Hyperopt, and Optuna.  For more details, see [Working with Tune Search Spaces](https://docs.ray.io/en/master/tune/tutorials/tune-search-spaces.html#tune-search-space-tutorial).  Deciding the best combination of search space and search algorithm is part of the art of being a Data Scientist and depends on the data, algorithm, and problem being solved!  

**Below, we define our search space is:**
- 2 different Scikit-learn algorithms 
- Some or all NYC taxi drop-off locations. 

**And we define the search algorithm is:**
- Grid search.

> This means every permutation of each algorithm and each NYC Taxi drop-off location will be a separate experiment!  

Ray Tune partitions the Search space using the specified Search algorithm and takes care of running a Tune job on each partition in parallel.  Specifically, Ray Tune will pass a config dictionary to each partition and make a Trainable function call.

In [8]:
# 1. Define a search space.
sample_locations = [145, 166, 152] if SMOKE_TEST else all_location_ids
search_space = {
    "model": tune.grid_search([LinearRegression(fit_intercept=True), 
                                DecisionTreeRegressor(max_depth=3)]),
    "location": tune.grid_search(sample_locations),
}

## Define a Trainable (callable) function <a class="anchor" id="define_trainable"></a>

🧪 Typically when you are running Data Science experiments, you want to be able to keep track of summary metrics for each experiment, so you can decide at the end which experiments were the best.  That way, you can decide which model to deploy.

📋 Ray Tune produces an experiments Summary table with metrics which you specify inside a "Trainable" or "callable" function.

<b>Define a "Trainable" object that can be called from every parallel Tune experiment.</b> 
>Ray Tune has two ways of defining a trainable, namely the [Function API](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#trainable-docs) and the Class API. Both are valid ways of defining a trainable, but *the Function API is generally recommended*.

**In the cell below, we define a "Trainable" function called `train_model()`**.  
- It takes as input a config dictionary argument. 
- The output can be a simple dictionary of metrics which will be reported back to the Tune Summary table.  In our case, we've chosen to force a checkpoint save of each model in addition to each experiment's metrics.
- `train_model()` will be called in parallel by Tune for each partition of the Tune search space.  
- Since we are using **grid search**, this means `train_model()` will be run *in parallel for every permutation* in the Tune search space!

In [9]:
# 2. Define a custom train function
def train_model(config: dict):

    model = config['model']
    the_location = config['location']
    
    # Load data.
    df_list = [read_data(f, the_location) for f in s3_files]   
    df_raw = pd.concat(df_list, ignore_index=True)
    df = transform_df(df_raw)
    
    # Train/valid split.
    train_df, valid_df = train_test_split(df, test_size=0.2, shuffle=True)
    train_X = train_df[["passenger_count", "trip_distance"]]
    train_y = train_df.trip_duration
    valid_X = valid_df[["passenger_count", "trip_distance"]]
    valid_y = valid_df.trip_duration

    # Train model.
    model = model.fit(train_X, train_y)
    pred_y = model.predict(valid_X)
    
    # Evaluate.
    error = sklearn.metrics.mean_absolute_error(valid_y, pred_y)
    
    # Save the model as a Tune Checkpoint.  
    # This is done through ray.air.session.report() API.
    # https://docs.ray.io/en/latest/tune/tutorials/tune-checkpoints.html
    
    # Define a model checkpoint.
    checkpoint = SklearnCheckpoint.from_estimator(model, path=".")

    # Save checkpoint and report back metrics, using ray.air.session.report()
    metrics = dict(error = error)
    session.report(
            metrics, 
            checkpoint=checkpoint)

## Run independent experiments in Parallel using Ray Tune <a class="anchor" id="run_tune_search"></a>

Now we are ready to use Ray Tune to run separate, parallel training jobs, each as a different experiment, per dropoff location.

**Configure the resources allocated per experiment (trial).** Tune uses this resources allocation to control the parallelism. For example, if each trial was configured to use 4 CPUs, and the cluster had only 32 CPUs, then Tune will limit the number of concurrent trials to 8 to avoid overloading the cluster. For more information, see [A Guide To Parallelism and Resources](https://docs.ray.io/en/master/tune/tutorials/tune-resources.html#tune-parallelism).

In [10]:
# 3. Can customize resources per experiment, here we set 1 CPU each.
train_model = tune.with_resources(train_model, {"cpu": 1})

**Below, we introduce AIR configs and run the experiment using `tuner.fit()`.** Tune will report on experiment status, and after the experiment finishes, you can inspect the results. 

Notice in the AIR config, we have specified a local directory `my_Tune_logs` for logging instead of the default `~/ray_results` directory. Learn more about logging Tune results at [How to configure logging in Tune](https://docs.ray.io/en/master/tune/tutorials/tune-output.html#tune-logging).

Tune can [retry failed experiments automatically](https://docs.ray.io/en/master/tune/tutorials/tune-stopping.html#tune-stopping-guide), as well as entire experiments.  This is necessary in case a node on your remote cluster fails (when running on a cloud such as AWS or GCP).

In [11]:
# Define a tuner object using Ray AIR Tuner API
tuner = tune.Tuner(
    train_model, 
    param_space=search_space,
    run_config=air.RunConfig(
        
        #redirect logs to relative path instead of default ~/ray_results/
        local_dir = "my_Tune_logs",
        name = "batch_tuning",

        # Set Ray Tune verbosity.  Print summary table only with levels 2 or 3.
        verbose=2,
        ),
)

# 4. Run the experiments with Ray Tune
start = time.time()
results = tuner.fit()
total_time_taken = time.time() - start

# Assemble Tune results into a pandas dataframe
results_df = results.get_dataframe()

# Print some training stats
print(f"Total number of models: {results_df.shape[0]}")
print(f"TOTAL TIME TAKEN: {total_time_taken:.2f} seconds")
best_result = results.get_best_result(metric="error", mode="min").config
print(f"Best result: {best_result}")

2022-11-09 11:17:51,926	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,location,model,iter,total time (s),error
train_model_34a03_00000,TERMINATED,127.0.0.1:27533,145,LinearRegression(),1,188.856,561.359
train_model_34a03_00001,TERMINATED,127.0.0.1:27541,166,LinearRegression(),1,185.675,392.353
train_model_34a03_00002,TERMINATED,127.0.0.1:27542,152,LinearRegression(),1,188.982,281.491
train_model_34a03_00003,TERMINATED,127.0.0.1:27543,145,DecisionTreeReg_64f0,1,154.691,212.417
train_model_34a03_00004,TERMINATED,127.0.0.1:27544,166,DecisionTreeReg_6670,1,186.074,272.838
train_model_34a03_00005,TERMINATED,127.0.0.1:27545,152,DecisionTreeReg_67f0,1,177.048,645.461


2022-11-09 11:20:30,968	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'model': DecisionTreeRegressor(max_depth=3)}


Trial train_model_34a03_00003 reported error=212.41666666666666,should_checkpoint=True with parameters={'model': DecisionTreeRegressor(max_depth=3), 'location': 145}.
Trial train_model_34a03_00003 completed. Last result: error=212.41666666666666,should_checkpoint=True


2022-11-09 11:20:53,331	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'model': DecisionTreeRegressor(max_depth=3)}


...

Trial train_model_34a03_00000 reported error=561.3589579264323,should_checkpoint=True with parameters={'model': LinearRegression(), 'location': 145}.
Trial train_model_34a03_00000 completed. Last result: error=561.3589579264323,should_checkpoint=True
Trial train_model_34a03_00002 reported error=281.4914916992187,should_checkpoint=True with parameters={'model': LinearRegression(), 'location': 152}.


2022-11-09 11:21:05,128	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'model': LinearRegression()}


Trial train_model_34a03_00002 completed. Last result: error=281.4914916992187,should_checkpoint=True


2022-11-09 11:21:05,255	INFO tune.py:758 -- Total run time: 193.33 seconds (192.58 seconds for the tuning loop).


Total number of models: 6
TOTAL TIME TAKEN: 193.36 seconds
Best result: {'model': DecisionTreeRegressor(max_depth=3), 'location': 145}


<br>

<b>After all the Tune experiments have run, pick the best model per dropoff location. </b>

We can assemble the Tune results ([ResultGrid object](https://docs.ray.io/en/master/tune/examples/tune_analyze_results.html)) into a pandas dataframe, then sort by minimum error, to select the best model per dropoff location.

In [12]:
# Assemble Tune results into a pandas dataframe
print(results_df.columns)
results_df = results_df[["config/location", "config/model", "error", "logdir"]]
results_df.rename(columns={'config/location':'location_id'}, inplace=True)
results_df.rename(columns={'config/model':'model_name'}, inplace=True)
results_df.set_index(["location_id", "model_name"], inplace=True)
results_df.head()

Index(['error', 'time_this_iter_s', 'should_checkpoint', 'done',
       'timesteps_total', 'episodes_total', 'training_iteration', 'trial_id',
       'experiment_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname',
       'node_ip', 'time_since_restore', 'timesteps_since_restore',
       'iterations_since_restore', 'warmup_time', 'config/location',
       'config/model', 'logdir'],
      dtype='object')


,,error,logdir
location_id,model_name,,
145,LinearRegression(),561.358958,/Users/christy/Documents/github_ray_temp/ray/d...
166,LinearRegression(),392.352924,/Users/christy/Documents/github_ray_temp/ray/d...
152,LinearRegression(),281.491492,/Users/christy/Documents/github_ray_temp/ray/d...
145,DecisionTreeRegressor(max_depth=3),212.416667,/Users/christy/Documents/github_ray_temp/ray/d...
166,DecisionTreeRegressor(max_depth=3),272.837566,/Users/christy/Documents/github_ray_temp/ray/d...


In [13]:
# Keep only 1 model per location_id with minimum error
final_df = results_df.reset_index()
final_df = final_df.loc[final_df.groupby('location_id')['error'].idxmin()].copy()
final_df.sort_values(by=["error"], inplace=True)
final_df.reset_index(inplace=True, drop=True)
print(final_df.dtypes)
final_df

location_id      int64
model_name      object
error          float64
logdir          object
dtype: object


,location_id,model_name,error,logdir
0,145,DecisionTreeRegressor(max_depth=3),212.416667,/Users/christy/Documents/github_ray_temp/ray/d...
1,166,DecisionTreeRegressor(max_depth=3),272.837566,/Users/christy/Documents/github_ray_temp/ray/d...
2,152,LinearRegression(),281.491492,/Users/christy/Documents/github_ray_temp/ray/d...


## Load a model from checkpoint and perform inference  <a class="anchor" id="load_checkpoint"></a>

> **[Ray AIR Predictors](https://docs.ray.io/en/latest/ray-air/predictors.html) make batch inference easy since they have internal logic to parallelize the inference.**

However, in this notebook, we will just restore a single scikit-learn model directly from checkpoint, and demonstrate it can be used for inference.  

Users can easily obtain checkpoint objects from the Tune results object. 

In [14]:
# Get a checkpoint path from Ray Tune results. 
checkpoint_path = final_df.logdir[0] + "/checkpoint_000000"
print(checkpoint_path)

# Recover SklearnCheckpoint object from path
checkpoint = SklearnCheckpoint.from_directory(checkpoint_path)
print(type(checkpoint))

# Restore a model from checkpoint
model = checkpoint.get_estimator()
print(type(model))

/Users/christy/Documents/github_ray_temp/ray/doc/source/ray-air/examples/my_Tune_logs/batch_tuning/train_model_34a03_00003_3_location=145,model=DecisionTreeRegressor_max_depth_3_2022-11-09_11-17-54/checkpoint_000000
<class 'ray.train.sklearn.sklearn_checkpoint.SklearnCheckpoint'>
<class 'sklearn.tree._classes.DecisionTreeRegressor'>


In [15]:
# Get a dropoff location
the_location = final_df.location_id[0]
the_location

145

In [16]:
# Create some test data
df_list = [read_data(f, the_location) for f in s3_files[:1]]   
df_raw = pd.concat(df_list, ignore_index=True)
df = transform_df(df_raw)

# Train/test split.
_, test_df = train_test_split(df, test_size=0.2, shuffle=True)
test_X = test_df[["passenger_count", "trip_distance"]]
test_y = np.array(test_df.trip_duration)  #actual values

In [17]:
# Inference using restored model from checkpoint
pred_y = model.predict(test_X)

# Zip together to visualize
pd.DataFrame(zip(pred_y, test_y), 
             columns = ["pred_y", "trip_duration"])

,pred_y,trip_duration
0,1272.750000,1496
1,370.000000,427
2,2671.500000,1918
3,1272.750000,1813
4,866.333333,1015
5,666.000000,649


<b>Compare validation and test error.</b>

During model training we reported error on "validation" data (random sample).  Below, we will report error on a pretend "test" data set (a different random sample).

Do a quick validation that both errors are reasonably close together.

In [18]:
# Evaluate.
error = sklearn.metrics.mean_absolute_error(test_y, pred_y)
print(f"Test error: {error}")

Test error: 289.9444444444444


In [19]:
# Compare test error with validation error
print(f"Train error: {final_df.error[0]}")

# Validation and test errors should be reasonably close together.

Train error: 212.41666666666663
